# Домашнее задание 2

**Кругликов А. С.**

## 1
Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
* взять в тренировочный набор турниры с dateStart из 2019 года; 
* в тестовый — турниры с dateStart из 2020 года.

In [90]:
import time
import pickle
import pandas as pd
import numpy as np
import math as mt
import torch
from torch import nn
from torch import optim
from scipy.stats import kendalltau, spearmanr
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

import warnings

warnings.filterwarnings('ignore')

In [2]:
results = pickle.load(open('results.pkl', 'rb'))
players = pickle.load(open('players.pkl', 'rb'))
tournaments = pickle.load(open('tournaments.pkl', 'rb'))

Взглянем на данные

In [6]:
results[1][0]

{'team': {'id': 242,
  'name': 'Команда Азимова',
  'town': {'id': 21, 'name': 'Баку'}},
 'mask': None,
 'current': {'name': 'Команда Азимова', 'town': {'id': 21, 'name': 'Баку'}},
 'questionsTotal': 0,
 'synchRequest': None,
 'position': 1,
 'controversials': [],
 'flags': [],
 'teamMembers': [{'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 476,
    'name': 'Анар',
    'patronymic': 'Беюкага оглы',
    'surname': 'Азимов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 878,
    'name': 'Фариз',
    'patronymic': 'Наим оглы',
    'surname': 'Аликишибеков'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 1872,
    'name': 'Аднан',
    'patronymic': 'Фариз оглы',
    'surname': 'Ахундов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'player': {'id': 13721,
    'name': 'Балаш',
    'patronymic': 'Алекпер оглы',
    'surname': 'Касумов'}},
  {'flag': None,
   'usedRating': 0,
   'rating': 0,
   'play

In [16]:
players[22799]

{'id': 22799,
 'name': 'Сергей',
 'patronymic': 'Игоревич',
 'surname': 'Николенко'}

In [39]:
tournaments[5527]

{'id': 5527,
 'name': 'Открытый чемпионат Мальты',
 'dateStart': '2019-04-06T15:00:00+03:00',
 'dateEnd': '2019-04-06T20:00:00+03:00',
 'type': {'id': 2, 'name': 'Обычный'},
 'season': '/seasons/52',
 'orgcommittee': [{'id': 25882,
   'name': 'Максим',
   'patronymic': 'Оскарович',
   'surname': 'Поташев'}],
 'synchData': None,
 'questionQty': {'1': 15, '2': 15, '3': 15}}

### Подготовка данных

In [40]:
train_data_list =  []
test_data_list = []
for tournament, teams in results.items():
    for team in teams:
        tournament_year = tournaments[tournament]["dateStart"][:4]
        if team.get("mask", None) and tournament_year in ['2019', '2020']:
            for member in team['teamMembers']:
                add = {}
                add['team_id'] = int(team['team']['id'])
                add['team_member'] = int(member['player']['id'])
                add['tournament_id'] = int(tournament)
                add['avg_score'] = team['mask'].count('1') / len(team['mask'])
                add = {
                    "tournament_id": tournament,
                    "tournament_name": tournaments.get(tournament).get("name"),
                    "team_id": team["team"]["id"],
                    "team_name": team["team"]["name"],
                    "player_id": member["player"]["id"],
                    "player_name": member["player"]["surname"] + " " + member["player"]["name"] + " " + member["player"]["patronymic"],
                    "position": team.get("position", None),
                }
                if tournament_year == '2019':
                    add["number_of_questions"] = sum(tournaments.get(tournament).get("questionQty").values())
                    add["questions_mask"] = team["mask"]
                    if add["number_of_questions"] == len(add["questions_mask"]):
                        train_data_list.append(add)
                elif tournament_year == '2020':
                    test_data_list.append(add)

In [41]:
test_data = pd.DataFrame(test_data_list)

data_by_question = []
for item in train_data_list:
    for _, result in enumerate(item["questions_mask"]):
        if result in ["0", "1"]:
            add = {
                "tournament_id": item["tournament_id"],
                "team_id": item["team_id"],
                "player_id": item["player_id"],
                "question": str(item["tournament_id"]) + "_" + str(_),
                "basic_label": int(result),
            }
            data_by_question.append(add)

## 2
Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
* повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
* в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
* для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.
___
Предположим:
* если команда не ответила на вопрос, то ни один игрок команды не ответил на вопрос
* если команда ответила на вопрос, то каждый игрок ответил на вопрос

Обучим модель, которая предскажет вероятность того, что некоторый игрок, ответил на некоторый вопрос.

In [111]:
df_by_question = pd.DataFrame(data_by_question).astype({"tournament_id": np.int32,
                                                       "team_id": np.int32,
                                                       "player_id": np.int32,
                                                       "basic_label": np.uint8})

encoder = OneHotEncoder()
train_data = encoder.fit_transform(df_by_question[["player_id", "question"]])
x = torch.sparse.FloatTensor(
    torch.LongTensor(np.vstack((train_data.tocoo().row, train_data.tocoo().col))),
    torch.FloatTensor(train_data.data)
)
y = torch.FloatTensor(df_by_question["basic_label"].values).reshape(-1, 1)

In [112]:
class LogisticRegression(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.lin = nn.Linear(n_features, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        return self.sigmoid(self.lin(x))
    
def train_step(model, x, y, lr, n_iter):
    model.lin.reset_parameters()
    criteria = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr)
    for i in range(n_iter):
        tstart = time.time()
        optimizer.zero_grad()
        loss = criteria(model(x), y)
        loss.backward()
        optimizer.step()
        tstop = time.time()
        time_part = int((tstop - tstart) * (n_iter - i))
        print(f"Эпоха {i}, до конца обучения {time_part // 60} минут {time_part % 60} секунд")

In [114]:
n_features = x.shape[1]
model = LogisticRegression(n_features)
train_step(model, x, y, lr=1, n_iter=100)

Эпоха 0, до конца обучения 4 минут 31 секунд
Эпоха 1, до конца обучения 3 минут 36 секунд
Эпоха 2, до конца обучения 3 минут 32 секунд
Эпоха 3, до конца обучения 3 минут 25 секунд
Эпоха 4, до конца обучения 3 минут 23 секунд
Эпоха 5, до конца обучения 3 минут 18 секунд
Эпоха 6, до конца обучения 3 минут 22 секунд
Эпоха 7, до конца обучения 3 минут 21 секунд
Эпоха 8, до конца обучения 3 минут 13 секунд
Эпоха 9, до конца обучения 3 минут 13 секунд
Эпоха 10, до конца обучения 3 минут 15 секунд
Эпоха 11, до конца обучения 3 минут 8 секунд
Эпоха 12, до конца обучения 3 минут 11 секунд
Эпоха 13, до конца обучения 3 минут 0 секунд
Эпоха 14, до конца обучения 3 минут 3 секунд
Эпоха 15, до конца обучения 2 минут 59 секунд
Эпоха 16, до конца обучения 3 минут 13 секунд
Эпоха 17, до конца обучения 2 минут 57 секунд
Эпоха 18, до конца обучения 2 минут 59 секунд
Эпоха 19, до конца обучения 2 минут 51 секунд
Эпоха 20, до конца обучения 2 минут 47 секунд
Эпоха 21, до конца обучения 2 минут 44 секунд
Э

## 3
Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
* предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
* в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Для самопроверки: у меня средняя корреляция Спирмена на тестовом множестве 2020 года во всех моделях, включая baselines, получалась порядка 0.7-0.8, а корреляция Кендалла — порядка 0.5-0.6. Если у вас корреляции вышли за 0.9 или, наоборот, упали ниже 0.3, скорее всего где-то баг.

Если мы хотим сравнить наши результаты с тестовым набором данных, мы можем рассчитать силу команды. Затем мы можем ранжировать их и сравнивать с реальными позициями в тестовом наборе.
Силу команды можно представить как вероятность того, что команда ответит на средний вопрос. Поскольку разные игроки в команде отвечают на вопрос независимо друг от друга, мы можем определить сильные стороны команды как:

$${P_{team_i} = 1 - \prod_{j = 1}^{N}(1 - \sigma(p_{ij} + q_{avg} + b))}$$
- $p_{ij}$ - сила j-го игрока в команде $i$ размера $N$
- $q_{avg}$ - средний вопрос
- $b$ - смещение

In [115]:
weights = [model.lin.weight.data[0].numpy(), model.lin.bias.data[0].numpy()]

In [116]:
def corr(data, encoder, parameters):
    player_weights = {}
    player_sum = 0
    player_count = 0
    question_sum = 0
    question_count = 0
    for i, feature in enumerate(encoder.get_feature_names()):
        if feature[:3] == "x0_":
            player_weights[int(feature[3:])] = parameters[0][i]
            player_sum += parameters[0][i]
            player_count += 1
        else:
            question_sum += parameters[0][i]
            question_count += 1
    
    data["player_weights"] = data["player_id"].apply(lambda x: player_weights.get(x))
    data["player_weights"].fillna(player_sum / player_count, inplace=True)
    data["player_probability"] = data["player_weights"].apply(lambda x: 1 / (1 + np.exp( - (x + question_sum / question_count + parameters[1]))))
    probas = data.groupby(["tournament_id", "team_id"])["player_probability"].apply(lambda x: 1 - np.prod(1 - x))
    position = data.groupby(["tournament_id", "team_id"])["position"].first()
    group_data = pd.concat([probas, position], axis=1)
    group_data.sort_values(["tournament_id", "player_probability"], ascending=[True, True], inplace=True)
    spearman = group_data.groupby("tournament_id").apply(lambda x: - spearmanr(x["position"], x["player_probability"]).correlation).mean()
    kendall = group_data.groupby("tournament_id").apply(lambda x: - kendalltau(x["position"], x["player_probability"]).correlation).mean()
    return spearman, kendall

In [117]:
spearman, kendall = corr(test_data, encoder, weights)

print('Корреляция Спирмена: ', spearman)
print('Корреляция Кендалла: ', kendall)

Корреляция Спирмена:  0.7859253342024026
Корреляция Кендалла:  0.6290059220969266


По самопроверке проходит

## 4
Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:

* предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
* разработайте EM-схему для обучения этой модели, реализуйте её в коде;
* обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.

Пусть $t_{i}$ ответ команды на $i$-й вопрос, $z_{ij}$ - скрытая переменная - ответ $j$-го игрока в $i$-й команде


${p(t = 1|z = 1) = 1}$

Тогда, на Е-шаге хотим:
${p(z_{ij} = 1|t_{i} = y)}$, где y - ответ.

Так как ${p(z_{ij} = 1|t_{i} = 0) = 0}$ нам нужно только найти:
$${p(z_{ij} = 1|t_{i} = 1) = \frac{p(t_{i} = 1|z_{ij} = 1)p(z_{ij} = 1)}{p(t_{i} = 1)}}$$

Используя предсказания М-шага:
$${p(z_{ij} = 1|t_{i} = 1) = \frac{\sigma(p_i + q_j + b)}{1 - \prod_{k=1}^{N}(1 - \sigma(p_{ik} + q_j + b))}}$$

In [118]:
def e_step(data, predicts):
    data["label"] = predicts
    data.loc[data["basic_label"] == 0, "label"] = 0
    data.loc[data["basic_label"] == 1, "label"] = data.loc[data["basic_label"] == 1, "label"] / data[data["basic_label"] == 1].groupby(["team_id", "question"])["label"].transform(lambda x: 1 - np.prod(1 - x.values))
    return data


def m_step(model, x, y, lr=0.001, n_iter=10):
    model.lin.reset_parameters()
    criteria = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr)
    for i in range(n_iter):
        optimizer.zero_grad()
        output = model(x)
        loss = criteria(output, y)
        loss.backward()
        optimizer.step()

In [119]:
predicts = model(x).detach().numpy().flatten()
question_answer_data__ = e_step(df_by_question.copy(), predicts)
y = torch.FloatTensor(question_answer_data__["label"].values).view(-1, 1)
m_step(model, x, y, lr=1, n_iter=100)
weights = [model.lin.weight.data[0].numpy(), model.lin.bias.data[0].numpy()]
spearman, kendall = corr(test_data, encoder, weights)
print("Шаг 1")
print(f"spearman {spearman}, kendall {kendall}")

predicts = model(x).detach().numpy().flatten()
for i in range(1, 6):
    question_answer_data__ = e_step(question_answer_data__, predicts)
    y = torch.FloatTensor(question_answer_data__["label"].values).view(-1, 1)
    m_step(model, x, y, lr=1, n_iter=100)
    predicts = model(x).detach().numpy().flatten()
    weights = [model.lin.weight.data[0].numpy(), model.lin.bias.data[0].numpy()]
    spearman, kendall = corr(test_data, encoder, weights)
    print(f"Шаг {i + 1}")
    print(f"spearman {spearman}, kendall {kendall}")

Шаг 1
spearman 0.7939539774763885, kendall 0.6383474097560201
Шаг 2
spearman 0.7939497779169516, kendall 0.6380750059147122
Шаг 3
spearman 0.7968207513546135, kendall 0.6403902431428277
Шаг 4
spearman 0.7972287419520179, kendall 0.6411298928341518
Шаг 5
spearman 0.7998719493922131, kendall 0.6448814253865159
Шаг 6
spearman 0.8014189185611772, kendall 0.6466026940488063


## 5
А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [120]:
question_weights = {}
for i, feature in enumerate(encoder.get_feature_names()):
    if feature[:3] == "x1_":
        question_weights[feature[3:]] = weights[0][i]
        
tournament_weights = question_answer_data__.groupby("tournament_id")["question"].apply(lambda x: np.mean([question_weights[q] for q in x])).sort_values().reset_index()["tournament_id"].apply(lambda x: tournaments[x]["name"])

In [121]:
tournament_weights.head(10)

0    Чемпионат Санкт-Петербурга. Первая лига
1                     Чемпионат Таджикистана
2                                Угрюмый Ёрш
3     Зеркало мемориала памяти Михаила Басса
4                         Воображаемый музей
5                 Синхрон высшей лиги Москвы
6               Первенство правого полушария
7            Чемпионат Мира. Этап 2 Группа С
8           Чемпионат Мира. Этап 2. Группа В
9              Мемориал памяти Михаила Басса
Name: tournament_id, dtype: object

In [122]:
tournament_weights.tail(10)

665            (а)Синхрон-lite. Лига старта. Эпизод V
666                             Школьная лига. I тур.
667                           Школьная лига. III тур.
668                                  Кубок Закарпатья
669                    Гран-при Славянки. Общий зачёт
670    Студенческий чемпионат Калининградской области
671                                  One ring - async
672                               Синхрон Лиги Разума
673                 Шестой киевский марафон. Асинхрон
674                            Асинхрон по South Park
Name: tournament_id, dtype: object